# Ejemplo de Redes Neuronales
---
## Integrantes:
* Alexis Báez
* Ricardo Baquero
* Wladimir Gualoto

In [ ]:
# Importar las librerías necesarias
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Descripción
La base de datos de entrenamiento contiene información de 2000 celulares. El propósito del análisis es determinar el rango de precio de un celular utilizando sus características.
### Rangos de precios:
* 0: costo bajo
* 1: costo medio
* 2: costo alto
* 3: costo muy alto

### Características de los celulares:
* id: ID
* battery_power: Cantidad total de energía de la batería en mAh
* blue: Posee bluetooth o no
* clock_speed: Velocidad del reloj del procesador
* dual_sim: Tiene soporte de doble sim
* fc: Mega pixeles de la cámara frontal
* four_g: Posee 4G o no
* int_memory: Memoria interna en Gigabytes
* m_dep: Profundidad del teléfono en  cm
* mobile_wt: Peso del celular
* n_cores: Número de núcleos del procesador
* pc: Mega pixeles de la cámara principal
* px_height: Resolución vertical en pixeles
* px_width: Resolución horizontal en pixeles
* ram: Memoria RAM en MB
* sc_h: Altura de la pantalla en cm
* sc_w: Anchura de la pantalla en cm
* talk_time: mayor cantidad de tiempo que la batería totalmente cargada funcionará
* three_g: Posee 3G o no
* touch_screen: Posee pantalla táctil o no
* wifi: Posee wifi o no

In [ ]:
# Cargar en memoria los datos de entrenamiento.
train_df = pd.read_csv("../input/mobile-price-classification/train.csv")
# Imprimir las primeras 5 filas. 
train_df.head()

In [ ]:
# Imprimir una descripción de los datos.
train_df.describe().T

## Visualización de datos


In [ ]:
# Gráfico de correlación
plt.figure(figsize=(14,10))
sns.heatmap(train_df.corr(), annot=True, fmt=".2f");

Examinando el gráfico, es evidente que existe una alta correlación entre la cantidad de memoria RAM y el rango de precio del dispositivo.

In [ ]:
# Gráfico de puntos RAM x Rango de precios
sns.pointplot(y="ram", x="price_range", data=train_df)

## Entrenamiento del modelo mediante Redes Neuronales
### Preprocesamiento de los datos

In [ ]:
# Convertir la tabla a un arreglo de numpy y separar la columna de rangos de precio
X = train_df.iloc[:,:20].values
y = train_df.iloc[:,20:21].values

#### Normalización de los datos

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
print(X[0])

#### Codificación de la clasificación en arreglos de 0 y 1

In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
y = ohe.fit_transform(y).toarray()
print('Arreglo codificado de 1s:')
print(y[0:5])

#### Separar datos de entrenamiento y prueba

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)

Se utilizará un 10% de los datos de entrenamiento como datos de prueba para mejorar el modelo.

### Construcción del Modelo

Para nuestra red neural, utilizamos dos capas ocultas de dimensión 12 y 16.

In [ ]:
# Librerías
import keras
from keras.models import Sequential
from keras.layers import Dense

# Red neural
model = Sequential()
model.add(Dense(16, input_dim=20, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(4, activation='softmax'))

**Sequential** especifica a la librería "Keras" que estamos creando un modelo secuencialmente y el resultado de cada capa son los argumentos de entrada de la siguiente.

**model.add** se utiliza para agregar una capa a nuestra red neural. Se debe especificar que tipo de capa necesitamos.
Utilizamos **Dense** para especificar que es una capa completamente conectada.

In [ ]:
# Compilar el modelo 
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Utilizamos como función "loss" categorical_crossentropy la cual especifica que tenemos múltiples clases.

#### Entrenar el modelo

In [ ]:
history = model.fit(X_train, y_train, validation_data = (X_test,y_test), epochs=100, batch_size=64)

Se requiere especificar: los datos de entrada, las etiquetas de los datos, los datos de validación y sus etiquetas, la cantidad de épocas (iteraciones) y el tamaño de lote (cuantos datos se prueban al mismo tiempo).

#### Probar el modelo
Utilizaremos los datos de prueba que fueron extraídos de los datos de entrenamiento. 

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
# Convertir las predicciones a sus respectivas etiquetas
def pred_to_label(y_pred):
    pred = list()
    for i in range(len(y_pred)):
        pred.append(np.argmax(y_pred[i]))
    return pred

In [ ]:
from sklearn.metrics import accuracy_score

pred = pred_to_label(y_pred)
    
# Convertir las etiquetas codificadas como arreglos de 0 y 1 a sus etiquetas originales
test = list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))
    

# Imprimir la precisión del modelo
a = accuracy_score(pred,test)
print(f'La precisión del modelo es del {a*100}%')

## Visualización del entrenamiento

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Precisión del Modelo')
plt.ylabel('Precisión')
plt.xlabel('Época')
plt.legend(['Entrenamiento', 'Prueba'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Pérdida del Modelo') 
plt.ylabel('Pérdida') 
plt.xlabel('Época') 
plt.legend(['Entrenamiento', 'Prueba'], loc='upper left') 
plt.show()

## Validación arbitraria

Resultado del modelo utilizando cualquier fila o datos arbitrarios.

In [ ]:
# Fila a probar
row_to_test = 3

In [ ]:
# Cargar en memoria los datos de prueba.
test_df = pd.read_csv("../input/mobile-price-classification/test.csv")
# Imprimir la fila que se utilizará para estimar el rango de precios
test_df.loc[[row_to_test]]

In [ ]:
test_x = test_df.iloc[:,1:21].values
test_x = sc.fit_transform(test_x)

sel_row = np.array([test_x[row_to_test]])

new_pred = model.predict(sel_row)
pred = pred_to_label(new_pred)
    
print(f'Rango de precios estimado: {pred[0]}')